## Eksploracja grupowania metodą K-Means w R z wykorzystaniem zasad Tidy data.

### [**Quiz przed wykładem**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

W tej lekcji nauczysz się tworzyć grupy za pomocą pakietu Tidymodels oraz innych pakietów z ekosystemu R (nazwiemy je przyjaciółmi 🧑‍🤝‍🧑), a także wykorzystując nigeryjski zestaw danych muzycznych, który zaimportowałeś wcześniej. Omówimy podstawy K-Means dla grupowania. Pamiętaj, że jak nauczyłeś się w poprzedniej lekcji, istnieje wiele sposobów pracy z grupami, a metoda, którą wybierzesz, zależy od Twoich danych. Spróbujemy K-Means, ponieważ jest to najczęściej stosowana technika grupowania. Zaczynajmy!

Pojęcia, które poznasz:

-   Ocena sylwetki (Silhouette scoring)

-   Metoda łokcia (Elbow method)

-   Inercja

-   Wariancja

### **Wprowadzenie**

[Grupowanie metodą K-Means](https://wikipedia.org/wiki/K-means_clustering) to metoda wywodząca się z dziedziny przetwarzania sygnałów. Służy do dzielenia i grupowania danych w `k grupy` na podstawie podobieństw ich cech.

Grupy można wizualizować jako [diagramy Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), które zawierają punkt (lub 'nasiono') oraz odpowiadający mu obszar.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika autorstwa Jen Looper</figcaption>


Grupowanie metodą K-Means obejmuje następujące kroki:

1.  Data scientist zaczyna od określenia pożądanej liczby grup, które mają zostać utworzone.

2.  Następnie algorytm losowo wybiera K obserwacji z zestawu danych, które będą początkowymi centrami grup (czyli centroidami).

3.  Następnie każda z pozostałych obserwacji jest przypisywana do najbliższego centroidu.

4.  Następnie obliczana jest nowa średnia dla każdej grupy, a centroid przesuwany jest do tej średniej.

5.  Po przeliczeniu centrów każda obserwacja jest ponownie sprawdzana, aby zobaczyć, czy może być bliżej innej grupy. Wszystkie obiekty są ponownie przypisywane, korzystając z zaktualizowanych średnich grup. Kroki przypisywania grup i aktualizacji centroidów są powtarzane iteracyjnie, aż przypisania grup przestaną się zmieniać (czyli gdy osiągnięta zostanie konwergencja). Zazwyczaj algorytm kończy działanie, gdy każda nowa iteracja powoduje znikomy ruch centroidów, a grupy stają się statyczne.

<div>

> Zauważ, że ze względu na losowość początkowych k obserwacji używanych jako centroidy początkowe, możemy uzyskać nieco różne wyniki za każdym razem, gdy stosujemy procedurę. Z tego powodu większość algorytmów używa kilku *losowych startów* i wybiera iterację z najniższym WCSS. Dlatego zdecydowanie zaleca się zawsze uruchamiać K-Means z kilkoma wartościami *nstart*, aby uniknąć *niepożądanego lokalnego optimum.*

</div>

Ta krótka animacja wykorzystująca [grafikę](https://github.com/allisonhorst/stats-illustrations) autorstwa Allison Horst wyjaśnia proces grupowania:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Grafika autorstwa @allison_horst</figcaption>



Podstawowe pytanie, które pojawia się w grupowaniu, brzmi: skąd wiadomo, na ile grup podzielić dane? Jednym z ograniczeń metody K-Means jest konieczność ustalenia `k`, czyli liczby `centroidów`. Na szczęście `metoda łokcia` pomaga oszacować dobrą wartość początkową dla `k`. Zaraz ją wypróbujesz.

### 

**Wymagania wstępne**

Zaczniemy dokładnie tam, gdzie skończyliśmy w [poprzedniej lekcji](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), gdzie analizowaliśmy zestaw danych, tworzyliśmy wiele wizualizacji i filtrowaliśmy zestaw danych do interesujących obserwacji. Koniecznie sprawdź!

Będziemy potrzebować kilku pakietów, aby ukończyć ten moduł. Możesz je zainstalować za pomocą: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternatywnie, poniższy skrypt sprawdza, czy masz wymagane pakiety do ukończenia tego modułu i instaluje je, jeśli brakuje niektórych.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Zaczynajmy!

## 1. Taniec z danymi: Wybierz 3 najpopularniejsze gatunki muzyczne

To podsumowanie tego, co zrobiliśmy w poprzedniej lekcji. Czas na analizę danych!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 To poszło świetnie!

## 2. Więcej eksploracji danych.

Jak czyste są te dane? Sprawdźmy wartości odstające za pomocą wykresów pudełkowych. Skupimy się na kolumnach numerycznych z mniejszą liczbą wartości odstających (chociaż można oczyścić dane z wartości odstających). Wykresy pudełkowe mogą pokazać zakres danych i pomóc w wyborze kolumn do analizy. Pamiętaj, że wykresy pudełkowe nie pokazują wariancji, która jest istotnym elementem dobrych danych do klastrowania. Zobacz [tę dyskusję](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) dla dalszych informacji.

[Wykresy pudełkowe](https://en.wikipedia.org/wiki/Box_plot) są używane do graficznego przedstawienia rozkładu danych `numerycznych`, więc zacznijmy od *wyboru* wszystkich kolumn numerycznych wraz z popularnymi gatunkami muzycznymi.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Zobacz, jak pomocnik wyboru `where` ułatwia to zadanie 💁? Odkryj inne podobne funkcje [tutaj](https://tidyselect.r-lib.org/).

Ponieważ będziemy tworzyć wykres pudełkowy dla każdej cechy numerycznej i chcemy uniknąć używania pętli, przeformatujmy nasze dane na *dłuższy* format, który pozwoli nam skorzystać z `facets` - podwykresów, z których każdy przedstawia jeden podzbiór danych.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Czas na coś dłuższego! Teraz pora na trochę `ggplots`! Jakiego `geom` użyjemy?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Easy-gg!

Teraz widzimy, że dane są trochę "szumne": obserwując każdą kolumnę jako wykres pudełkowy, można zauważyć wartości odstające. Można by przejrzeć zestaw danych i usunąć te wartości odstające, ale wtedy dane stałyby się dość ograniczone.

Na razie wybierzmy, które kolumny wykorzystamy do naszego ćwiczenia grupowania. Wybierzmy kolumny numeryczne o podobnych zakresach. Moglibyśmy zakodować `artist_top_genre` jako wartość numeryczną, ale na razie ją pominiemy.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Obliczanie klastrowania k-means w R

Możemy obliczyć k-means w R za pomocą wbudowanej funkcji `kmeans`, zobacz `help("kmeans()")`. Funkcja `kmeans()` przyjmuje ramkę danych, w której wszystkie kolumny są numeryczne, jako swój główny argument.

Pierwszym krokiem przy użyciu klastrowania k-means jest określenie liczby klastrów (k), które zostaną wygenerowane w ostatecznym rozwiązaniu. Wiemy, że w zbiorze danych wyodrębniliśmy 3 gatunki muzyczne, więc spróbujmy z 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Obiekt kmeans zawiera kilka informacji, które są dobrze wyjaśnione w `help("kmeans()")`. Na razie skupmy się na kilku z nich. Widzimy, że dane zostały podzielone na 3 klastry o rozmiarach 65, 110, 111. Wynik zawiera również centra klastrów (średnie) dla 3 grup w odniesieniu do 5 zmiennych.

Wektor klastrowania to przypisanie klastra dla każdej obserwacji. Użyjmy funkcji `augment`, aby dodać przypisanie klastra do oryginalnego zestawu danych.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Świetnie, właśnie podzieliliśmy nasz zbiór danych na 3 grupy. Jak dobrze wypadło nasze klastrowanie 🤷? Spójrzmy na `Silhouette score`.

### **Silhouette score**

[Analiza silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) może być używana do badania odległości separacyjnej między powstałymi klastrami. Wynik ten waha się od -1 do 1, a jeśli wynik jest bliski 1, oznacza to, że klaster jest gęsty i dobrze oddzielony od innych klastrów. Wartość bliska 0 oznacza nakładające się klastry, w których próbki znajdują się bardzo blisko granicy decyzyjnej sąsiednich klastrów. [źródło](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metoda średniej silhouette oblicza średnią wartość silhouette dla różnych wartości *k*. Wysoki średni wynik silhouette wskazuje na dobre klastrowanie.

Funkcja `silhouette` w pakiecie cluster służy do obliczania średniej szerokości silhouette.

> Silhouette można obliczyć za pomocą dowolnej [metryki odległości](https://en.wikipedia.org/wiki/Distance "Distance"), takiej jak [odległość euklidesowa](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") lub [odległość Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), które omawialiśmy w [poprzedniej lekcji](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Nasz wynik to **.549**, czyli dokładnie w środku. Wskazuje to, że nasze dane nie są szczególnie dobrze dopasowane do tego rodzaju klastrowania. Zobaczmy, czy możemy potwierdzić tę intuicję wizualnie. Pakiet [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) udostępnia funkcje (`fviz_cluster()`), które pozwalają na wizualizację klastrowania.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Nakładanie się klastrów wskazuje, że nasze dane nie są szczególnie dobrze dopasowane do tego rodzaju klastrowania, ale kontynuujmy.

## 4. Określenie optymalnej liczby klastrów

Podstawowe pytanie, które często pojawia się w klastrowaniu metodą K-Means, brzmi: bez znanych etykiet klas, skąd wiadomo, na ile klastrów podzielić dane?

Jednym ze sposobów, aby to sprawdzić, jest użycie próbki danych do `utworzenia serii modeli klastrowania` z rosnącą liczbą klastrów (np. od 1 do 10) i ocena metryk klastrowania, takich jak **Silhouette score.**

Określmy optymalną liczbę klastrów, obliczając algorytm klastrowania dla różnych wartości *k* i oceniając **Sumę Kwadratów Wewnątrz Klastra** (WCSS). Całkowita suma kwadratów wewnątrz klastra (WCSS) mierzy zwartość klastrowania, a zależy nam na tym, aby była jak najmniejsza – niższe wartości oznaczają, że punkty danych są bliżej siebie.

Przeanalizujmy wpływ różnych wyborów `k`, od 1 do 10, na to klastrowanie.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Teraz, gdy mamy całkowitą sumę kwadratów wewnątrz klastrów (tot.withinss) dla każdego algorytmu klastrowania z centrum *k*, używamy [metody łokcia](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) do znalezienia optymalnej liczby klastrów. Metoda polega na wykreśleniu WCSS jako funkcji liczby klastrów i wybraniu [punktu łokcia krzywej](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") jako liczby klastrów do użycia.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Wykres pokazuje znaczną redukcję WCSS (czyli większą *spójność*) wraz ze wzrostem liczby klastrów z jednego do dwóch, a następnie zauważalną redukcję z dwóch do trzech klastrów. Po tym redukcja jest mniej wyraźna, co skutkuje `łokciem` 💪 na wykresie przy około trzech klastrach. To dobry znak, że istnieją dwa do trzech stosunkowo dobrze oddzielonych klastrów punktów danych.

Możemy teraz przejść do wyodrębnienia modelu klastrowania, gdzie `k = 3`:

> `pull()`: używane do wyodrębnienia pojedynczej kolumny
>
> `pluck()`: używane do indeksowania struktur danych, takich jak listy


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Świetnie! Przejdźmy do wizualizacji uzyskanych klastrów. Masz ochotę na trochę interaktywności z użyciem `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Być może spodziewalibyśmy się, że każdy klaster (reprezentowany przez różne kolory) będzie miał odrębne gatunki (reprezentowane przez różne kształty).

Przyjrzyjmy się dokładności modelu.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Dokładność tego modelu nie jest zła, ale też nie jest świetna. Może się okazać, że dane nie nadają się dobrze do grupowania metodą K-Means. Dane są zbyt niezrównoważone, zbyt słabo skorelowane, a różnice między wartościami w kolumnach są zbyt duże, aby dobrze je pogrupować. W rzeczywistości grupy, które się tworzą, są prawdopodobnie mocno wpływane lub zniekształcone przez trzy kategorie gatunków, które zdefiniowaliśmy powyżej.

Mimo to, była to całkiem pouczająca lekcja!

W dokumentacji Scikit-learn można zauważyć, że model taki jak ten, z grupami niezbyt dobrze wyodrębnionymi, ma problem z "wariancją":

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika z Scikit-learn</figcaption>



## **Wariancja**

Wariancja jest definiowana jako "średnia kwadratów różnic od średniej" [źródło](https://www.mathsisfun.com/data/standard-deviation.html). W kontekście tego problemu grupowania odnosi się do danych, w których wartości naszego zbioru danych mają tendencję do zbytniego odchylenia od średniej.

✅ To świetny moment, aby zastanowić się nad wszystkimi sposobami, w jakie można rozwiązać ten problem. Może warto trochę zmodyfikować dane? Użyć innych kolumn? Zastosować inny algorytm? Podpowiedź: Spróbuj [skalować dane](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), aby je znormalizować i przetestować inne kolumny.

> Wypróbuj ten '[kalkulator wariancji](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', aby lepiej zrozumieć ten koncept.

------------------------------------------------------------------------

## **🚀Wyzwanie**

Poświęć trochę czasu na pracę z tym notebookiem, zmieniając parametry. Czy możesz poprawić dokładność modelu, bardziej oczyszczając dane (na przykład usuwając wartości odstające)? Możesz użyć wag, aby nadać większe znaczenie wybranym próbom danych. Co jeszcze możesz zrobić, aby stworzyć lepsze grupy?

Podpowiedź: Spróbuj skalować dane. W notebooku znajduje się zakomentowany kod, który dodaje standardowe skalowanie, aby kolumny danych bardziej przypominały siebie nawzajem pod względem zakresu. Zauważysz, że chociaż wynik silhouette score spada, "załamanie" na wykresie łokcia wygładza się. Dzieje się tak, ponieważ pozostawienie danych nieskalowanych pozwala danym o mniejszej wariancji mieć większy wpływ. Przeczytaj więcej o tym problemie [tutaj](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Quiz po wykładzie**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Przegląd i samodzielna nauka**

-   Sprawdź symulator K-Means [taki jak ten](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Możesz użyć tego narzędzia, aby wizualizować przykładowe punkty danych i określić ich centroidy. Możesz edytować losowość danych, liczbę grup i liczbę centroidów. Czy pomaga to zrozumieć, jak dane mogą być grupowane?

-   Zajrzyj również do [tego materiału o K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) ze Stanfordu.

Chcesz wypróbować swoje nowo zdobyte umiejętności grupowania na zestawach danych, które dobrze nadają się do grupowania metodą K-Means? Zobacz:

-   [Trenuj i oceniaj modele grupowania](https://rpubs.com/eR_ic/clustering) za pomocą Tidymodels i innych narzędzi

-   [Analiza grup K-Means](https://uc-r.github.io/kmeans_clustering), Przewodnik UC Business Analytics R Programming

- [Grupowanie K-Means z zasadami tidy data](https://www.tidymodels.org/learn/statistics/k-means/)

## **Zadanie**

[Wypróbuj różne metody grupowania](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## PODZIĘKOWANIA DLA:

[Jen Looper](https://www.twitter.com/jenlooper) za stworzenie oryginalnej wersji tego modułu w Pythonie ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) za stworzenie niesamowitych ilustracji, które sprawiają, że R jest bardziej przyjazny i angażujący. Znajdź więcej ilustracji w jej [galerii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Miłej nauki,

[Eric](https://twitter.com/ericntay), Złoty Ambasador Studentów Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Ilustracja autorstwa @allison_horst</figcaption>



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
